# 0 Imports

In [1]:
import requests
import zipfile
import io
import os

# 1 Get data

In [3]:
api_key = os.getenv('API_KEY')

In [2]:
# URL of the CSV file
url = f'https://api.os.uk/downloads/v1/products/OpenUPRN/downloads?area=GB&format=CSV&redirect&key={api_key}'

# Fetch the CSV data from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the ZIP data
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall('extracted_files')
    print("ZIP file downloaded and extracted successfully.")
else:
    print(f"Failed to fetch data: {response.status_code}")


ZIP file downloaded and extracted successfully.


# 2 Upload data